# Example of RAG using LangChain with HANA Vector DB and Generative AI Hub SDK
This notebook shows a short example of how a RAG application could be built using LangChain, HANA Vector DB and Generative AI Hub SDK.

## Setup

Switching the current directory of the notebook to the repo root

In [1]:
import os
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

/sapmnt/home/I574161/projects/rag-langchain-hana-example/notebooks
/sapmnt/home/I574161/projects/rag-langchain-hana-example


Importing required libraries

In [2]:
import pandas as pd

import json

from hdbcli import dbapi
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model, init_llm
from langchain_community.vectorstores import HanaDB
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

Loading required secrets

In [3]:
with open("secrets/gen-ai-hub-service-key.json", "r") as f:
    gen_ai_hub_service_key = json.load(f)

with open("secrets/hana-secrets.json", "r") as f:
    hana_secrets = json.load(f)

Setting up connection to HANA DB

In [4]:
hana_conn = dbapi.connect(
    address=hana_secrets["host"],
    port=hana_secrets["port"],
    user=hana_secrets["user"],
    password=hana_secrets["password"],
    autocommit=True,
    sslTrustStore=hana_secrets["certificate"],
)

Configuring connection to Generative AI Hub

**Note:** AI Core resource group might or might not be in `gen_ai_hub_service_key["appname"]`, so you might need to check via AI Core API or AI Launchpad for the correct value

In [5]:
os.environ["AICORE_AUTH_URL"] = gen_ai_hub_service_key["url"]
os.environ["AICORE_CLIENT_ID"] = gen_ai_hub_service_key["clientid"]
os.environ["AICORE_CLIENT_SECRET"] = gen_ai_hub_service_key["clientsecret"]
os.environ["AICORE_RESOURCE_GROUP"] = gen_ai_hub_service_key["appname"].split("!")[0]
os.environ["AICORE_BASE_URL"] = f"{gen_ai_hub_service_key['serviceurls']['AI_API_URL']}/v2"

proxy_client = get_proxy_client("gen-ai-hub")

Displaying a list of models available in Generative AI Hub

In [6]:
proxy_client.deployments

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d1e30862f24f01ec', config_id='cb08ab6d-94d9-4534-a60e-922ec1be66ff', config_name='gemini-1.0-pro-config-1', deployment_id='d1e30862f24f01ec', model_name='gemini-1.0-pro', created_at=datetime.datetime(2024, 4, 27, 5, 34, 58, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'gcp-vertexai', 'model_version': '001'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d50f02e66f040e9f', config_id='2f34dd34-eb58-482d-a1c2-d1450011ac88', config_name='chat-bison-config-1', deployment_id='d50f02e66f040e9f', model_name='chat-bison', created_at=datetime.datetime(2024, 4, 27, 5, 34, 56, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'gcp-vertexai', 'model_version': '002'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com

## RAG Application

Defining embeddings and HANA Vector DB client objects from LangChain

In [7]:
embeddings = init_embedding_model("text-embedding-ada-002", proxy_client=proxy_client)
hana_vectordb = HanaDB(embedding=embeddings, connection=hana_conn, table_name="RAG_EXAMPLE_VECTORSTORE")

Reading data from CSV file, and writing texts and corresponding vectors to HANA DB

In [8]:
input_df = pd.read_csv("data/rag_example_inputs.csv")
loader = DataFrameLoader(data_frame=input_df, page_content_column="text")
documents_to_index = loader.load()
hana_vectordb.add_documents(documents_to_index)

[]

Building RAG chain with LangChain Expression Language (LCEL)

In [9]:
hana_vector_retriever = hana_vectordb.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context": hana_vector_retriever,
        "question": RunnablePassthrough(),
    }
)

llm = init_llm("gpt-4", proxy_client=proxy_client)

rag_chain = setup_and_retrieval | prompt | llm | output_parser

Running RAG chain with different queries

In [10]:
questions = [
    "What is Python typically used for?",
    "What is JavaScript typically used for?",
    "Which programming languages are dynamically typed?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"RAG Answer: {rag_chain.invoke(question)}")
    print()

Question: What is Python typically used for?


RAG Answer: The text does not provide specific information on what Python is typically used for.

Question: What is JavaScript typically used for?
RAG Answer: JavaScript is typically used as a core technology of the Web, alongside HTML and CSS. It is used on the client side for webpage behavior in 99% of websites. JavaScript also has application programming interfaces (APIs) for working with text, dates, regular expressions, standard data structures, and the Document Object Model (DOM).

Question: Which programming languages are dynamically typed?
RAG Answer: The programming languages that are dynamically typed are Ruby and Python.

